##  Defining new types

In this notebook, we will look at how to define new types in Julia. This is very common when you need a certain **kind (type) of behaviour** that is not captured by the "built-in" types.

Note that types that you define are, in fact, on the same footing as the "built-in" types in the standard library (`Int`, `Float64`, etc.), which are themselves defined in Julia code.

## A wrapper type

We will look at a concrete example that arose naturally in real code in the author's pacakage 
[`IntervalConstraintProgramming.jl`](https://github.com/dpsanders/IntervalConstraintProgramming.jl): defining a type  to represent the volume of an object. This is similar to a [post on the `julia-users` mailing list](https://groups.google.com/forum/#!searchin/julia-users/probability/julia-users/PpXLxHajsfA/UWpmJZd2BQAJ) about defining a type to represent a probability.  

Despite the fact that the type contains only a single field, many of the issues that confuse people can be appreciated in this apparently simple situation.

We could choose to represent a volume with a standard `Float64`, but, as we shall see, there are good reasons to wrap this into a new type:

In [ ]:
immutable Vol
    value::Float64
end

We can now create an object of type `Vol` with

In [ ]:
V = Vol(3)

Let's recall how to look inside the object. We can do it interactively with `V.<TAB>` [press the `TAB` key after typing `V.` at the REPL, in the notebook, or in Atom].

Alternatively, we can use `fieldnames`, which returns an array of `Symbol`s, containing the names of the fields:

In [ ]:
fieldnames(V)

We access the field with any of the following commands:

In [ ]:
V.value

In [ ]:
getfield(V, :value)  # by name

In [ ]:
getfield(V, 1)       # by number

If the type were declared with `type`, instead of `immutable`, then we could also change the value via `setfield`.
This gives an error for an immutable object:

In [ ]:
V.value = 4

## Constructors

So far, we have been using the **default constructors**. Recall that the constructors for a type are functions that create objects of that type. We can see which constructors are available using `methods`:

In [ ]:
methods(Vol)

The output from Julia v0.5 is clearer:

    # 3 methods for generic function "(::Type)":
    Vol(value::Float64) at REPL[1]:2
    Vol(value) at REPL[1]:2
    (::Type{T}){T}(arg) at sysimg.jl:60

The problem is that we can do the following:

In [ ]:
Vol(-1)

This should not be allowed, since volumes cannot be negative. We want to impose a constraint *at the moment when we  create the object*, i.e. in the constructor. To do so, we define an **inner constructor** inside the type definition.

Since Julia does not allow redefining types, we need to clear the workspace before we do so:

In [ ]:
workspace()

In [ ]:
immutable Vol
    value::Float64
    
    function Vol(x)
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        new(x)
    end
end

Here we have used the so-called ["short-circuit" evaluation](http://docs.julialang.org/en/release-0.4/manual/control-flow/#man-short-circuit-evaluation) operator `&&`, which can be thought of as an `if...then`.

The function `new` is a special function that actually instantiates the object. If the type has several fields, some may be left empty, by not including them in the call to `new`, and filled in later (if the type is not `immutable`).

In [ ]:
Vol(3)

In [ ]:
Vol(-1)

## Making types parametric

There is no reason why volumes should be floating-point numbers -- they may be any kind of number. We could just remove the type annotation:

In [ ]:
workspace()

In [ ]:
immutable Vol
    value   # DO NOT DO THIS!
end

but we should **never do this**.  [By "never", we of course mean "only if you are really sure that you know what you're doing" and won't complain about the consequences.] The reason for this is that Julia is unable to infer the type of the variable, and so will be **very slow**.

Instead, what we really mean is that there should be a *different type* of `Vol` for each type that it could contain - integers, floats, `BigFloat`s, rationals, intervals, etc.
To tell Julia this, we use a **type parameter**, written with curly braces:

In [ ]:
workspace()

In [ ]:
immutable Vol{T}
    value::T   # DO THIS INSTEAD!
end

In words, this says "for **any** type `T`, a `Vol` with type parameter `T` looks like the following". This makes `Vol` a **parametric type**, a kind of template. This turns out to be a surprisingly powerful concept when combined with multiple dispatch.

Let's try this out:

In [ ]:
Vol(3)

In [ ]:
Vol(3.5)

In [ ]:
Vol("hello")

Here we find a new problem: the volume should be a real number. We can specify this restriction:

In [ ]:
workspace()

In [ ]:
immutable Vol{T<:Real}  # only accept types T that are subtypes of Real
    value::T   
end

In [ ]:
Vol("hello")

In [ ]:
Vol(3)

Let's put back the inner constructor:

In [ ]:
workspace()

In [ ]:
immutable Vol{T<:Real}
    value::T   
    
    function Vol(x)
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        new(x)
    end
end

In [ ]:
Vol(3)

Suddenly we can no longer construct objects of our new type! This turns out to be due to the fact that defining an inner constructor prevents the usual default constructors from being defined. We will fix this in a minute by defining them ourselves.

We *can* still construct an object, but we have to explicitly specify the type:

In [ ]:
Vol{Int}(3)

This is a pain and undesirable -- the point of Julia is that *it* is supposed to infer the types for us. 

The solution is to analyse what we want Julia to do, and then tell Julia to do exactly that. 

We would like Julia to look at the type of the object `x` passed to the constructor - call it `T` - and automatically use the constructor *for `Vol{T}`*, passing `x` as an argument, as we just did in the explicit call.

The syntax to do this looks strange at first (and second) glance:

In [ ]:
Vol{T}(x::T) = Vol{T}(x)

Now it works:

In [ ]:
Vol(3)

What is going on here? We seem to almost be repeating ourselves, writing the same both on the left and right of the assignment operator `=`. Once again, we are telling Julia to treat `T` as a type parameter. However, the meaning of `{T}` is *different* on the two sides.

- The left-hand side means: "for **any** type `T`, define a function `Vol` that takes a parameter `x` *of type `T`*".
This can be thought of as a specification for a matching algorithm: look at the function call and see if it matches this "template".


- The assignment operator has its usual meaning: "define the function on the left by the expression on the right".


- The right-hand side means: "call the constructor `Vol{T}` with argument `x`".

The point is that by the time the right-hand side is evaluated, Julia has **already realised** what `T` must be, since the function definition is matched by a call to `Vol` with an argument `x` of *any* type `T`!

The parametrised definition can be thought of as defining a "potentially infinite" number of different functions. Each of them is only instantiated, however, when the function is actually called with an argument of the corresponding type.

Furthermore, Julia is sophisticated enough that we can make multiple definitions like this. For example, we may want to store all volumes that are integers as exact integers, but all floating-point volumes as `BigFloat`s. We can do this by making a special method for floats:

In [ ]:
Vol{T<:AbstractFloat}(x::T) = Vol{BigFloat}(x)

In [ ]:
Vol(3.5)

Passing in integers does *not* match this new definition, and so still uses the previous definition:

In [ ]:
Vol(3)

## Arithmetic

We would now like to define arithmetic operations:

In [ ]:
V1 = Vol(3)
V2 = Vol(4)

In [ ]:
V1 + V2

Since we have not specified that `+` means for our type, Julia cannot know what to do.

We may, for some reason, want to define the sum of volumes only if they contain of the same type. We can follow the idea and syntax discussed for the constructor:

In [ ]:
import Base.+

+{T}(V1::Vol{T}, V2::Vol{T}) = Vol{T}(V1.value + V2.value)

This looks a bit noisy, but we just have to parse it piece by piece: it is a method of `+`, that is parametric with one type parameter `T`, and whose arguments are `Vol`s parametrised by **the same** `T`. This is important, since it means that the definition **will not match** function calls with two different types. This is one aspect of **multiple dispatch**, i.e. the fact that Julia takes into account the types of *all* arguments to determine which method is called.

In [ ]:
V1 + V2

However, this may be too restrictive; for example, we may wish to be able to add volumes with different integer types inside:

In [ ]:
Vol{Int64}(1) + Vol{BigInt}(3)

We can again define a more specialised method:

In [ ]:
+{T1<:Integer,T2<:Integer}(V1::Vol{T1}, V2::Vol{T2}) = Vol{promote_type(T1,T2)}(V1.value + V2.value)

[We could have left out the type parameter on the right-hand side, but this leads to a method ambiguity warning, in which two different methods match the combination of argument types, in this case when we have two equal integer types.]

Here, we have *two* type parameters that are both subtypes of the abstract `Integer` type, and `promote_type(T,S)` is a function from `Base` that determines the smallest supertype of both `T` and `S` that is able to correctly represent values of both types.

In [ ]:
Vol{Int64}(1) + Vol{BigInt}(3)

Note that sometimes such a redefinition will apparently not work, since the previous definition, *even if it threw an error*, will be cached by Julia. In this case, it will be necessary to clear the workspace first and redefine everything. This can soon become tiresome and may be partially solved by moving the definitions out into a file, and `include`ing the file after calling `workspace()` to define the types. This kind of workflow discussion is best decided in practice.

## Adding more parameters 

Types may be parametrized by more than one parameter. In the case of volumes, we should not be able to "add" a 1-dimensional volume (length) to a 2-dimensional one (area), since this makes no sense physically or mathematically; on the other hand, we can multiply them to get a 3-dimensional volume. We thus need to add an integer parameter to the `Vol` type:

In [ ]:
workspace()

In [ ]:
immutable Vol{N,T<:Real}
    value::T   
end

Again, a default constructor is created, but now we must explicitly specify the types, since the type `N` cannot be inferred from the argument:

In [ ]:
Vol{3, Float64}(3)

represents a three-dimensional volume of magnitude 3.

We wish to define the sum *only* when the dimensions are the same:

In [ ]:
import Base.+

+{N,T}(V1::Vol{N,T}, V2::Vol{N,T}) = Vol{N,T}(V1.value + V2.value)

This will match only when both arguments have the same parameters `T` and `N`. We could also promote the types:

In [ ]:
+{N,T1,T2}(V1::Vol{N,T1}, V2::Vol{N,T2}) = Vol{N,promote_type(T1,T2)}(V1.value + V2.value)

We make a useful outer constructor that can infer the type of its argument, but still needs an explicit dimension:

In [ ]:
Vol{T}(N, x::T) = Vol{N,T}(x)

We would also like to display objects in a nicer way:

In [ ]:
import Base.show
show{N,T}(io::IO, x::Vol{N,T}) = print(io, N, "-dimensional volume with value ", x.value)

In [ ]:
A = Vol(2, 3.)   # volume of dimension 2 with value 3.0

In [ ]:
ℓ = Vol(1, 10.)  # write as \ell<TAB>

In [ ]:
A + ℓ

It is not nice for the user to throw this slightly cryptic error message. We can provide a better error message when volumes of different dimension are added; we should check that volumes of the same dimension *still works*. Such things should be codified in unit test suites.

In [ ]:
workspace()

In [ ]:
immutable Vol{N,T<:Real}
    value::T   
end

Vol{T}(N, x::T) = Vol{N,T}(x)

import Base.+

+{N,T}(V1::Vol{N,T}, V2::Vol{N,T}) = Vol{N,T}(V1.value + V2.value)
+{N1,N2,T}(V1::Vol{N1,T}, V2::Vol{N2,T}) = throw(ArgumentError("Volumes of different dimension cannot be added"))

In [ ]:
V1 = Vol(3, 3)
V2 = Vol(3, 4)
V1 + V2

In [ ]:
Vol(3, 3) + Vol(2, 4)

**Exercise**
Define the product (`*`) of any two objects of type `Vol`.

## Conversion

We may want to define functions like `sin` that act on objects of our new type.
[We may *not* want to define such functions, on the other hand, since we could argue that a `Vol` is not a real number, but rather is a combination of a real number together with a dimension.]
Let's try it:

In [ ]:
sin(V1)

Of course, Julia doesn't know how to do this, since we haven't defined it.
However, an object of type `Vol` is just a simple wrapper around a number, so we could just say

In [ ]:
import Base.sin
sin(x::Vol) = sin(x.value)

In [ ]:
sin(V1)

It is painful / boring / prone to error to do this for every function (although we will see in the Metaprogramming notebook how to do streamline this). An alternative is to tell Julia explicitly that a `Vol` is actually a kind of real number, i.e. a subtype of `Real`. [It is sometimes the case that this is mathematically not true, but it is a convenience to simplify the code, for example for interval subsets of the real line.]

In [ ]:
workspace()

In [ ]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

In [ ]:
V = Vol{3,Float64}(3)

In [ ]:
sin(V)

We now get a different error. We can see which method Julia is now using:

In [ ]:
@which sin(V)

This is a generic "fallback" ("catch-all") method, which is called when no more specialised method has been defined.

Following the link to the Julia source code, which is written in Julia and therefore amazingly understandable (with some work), and if we understand some metaprogramming, we find that the definition of `sin(x::Real)` is

    sin(x::Real) = sin(float(x))

We could just define `float` for our type, which is supposed to convert an object to it's most natural floating-point equivalent:

In [ ]:
workspace()

In [ ]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

import Base.float
float(V::Vol) = float(V.value)

In [ ]:
V = Vol{3,Float64}(3)
sin(V)

However, in fact `float` is already defined for *any* `x::Real`:

In [ ]:
workspace()

In [ ]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

In [ ]:
V = Vol{3,Float64}(3)

In [ ]:
@which float(V)

The definition reads:

    float(x) = convert(AbstractFloat, x)

Finally, we see that rather we need to investigate Julia's capabilities for [conversion and promotion](http://docs.julialang.org/en/release-0.4/manual/conversion-and-promotion/) and define how to `convert` an object of our type into a float:

In [ ]:
import Base.convert
convert(::Type{AbstractFloat}, x::Vol) = float(x.value)

Now any standard unary function works:

In [ ]:
sin(V), cos(V), exp(V)

In [ ]:
V2 = Vol{3, BigFloat}(big"3.1")

`::Type{AbstractFloat}` refers to an argument which has type "type of `AbstractFloat`"; the only object that has that type is `AbstractFloat` itself; this thus matches function calls of the form 

    convert(AbstractFloat, V)

In [ ]:
convert(AbstractFloat, V2)

## Promotion 

In the case of `Vol`, it is not natural to define all arithmetic operations, so let's simplify even further and just wrap a `Float64` in a `Prob` type, representing a probability. [We will assume that it makes sense to define all arithmetic operations here, although in a given application that may not be the case.]

In [ ]:
workspace()

In [ ]:
immutable Prob{T<:AbstractFloat} <: Real
    _::T
    
    function Prob(x)
        (!(0 <= x <= 1)) && throw(ArgumentError("Probability must be between 0 and 1."))
        new(x)
    end
end

Prob{T<:AbstractFloat}(x::T) = Prob{T}(x)

[Note that `_` is a valid variable name in Julia, and is often used for a variable whose name we don't care about.]

In [ ]:
x = Prob(0.5)

We see that multiplying `x` by a real number does not give a `MethodError`:

In [ ]:
3x

since a `Vol` is a subtype of `Number`. 

Rather, Julia is trying to use its promotion machinery, in which such operations are handled by **promoting** both arguments to the same type:

In [ ]:
@which 3x

We can see the process using `@which` (or using the debugger in Julia v0.5):

In [ ]:
@which promote(3, x)

In [ ]:
@which promote_type(Int, Prob{Float64})

which is defined in terms of `promote_rule(T,S)`, e.g.

In [ ]:
@which promote_rule(Float64, Int)

So if we want to have promotion work, we need to define `promote_rule`:

In [ ]:
workspace()

In [ ]:
immutable Prob{T<:AbstractFloat} <: Real
    _::T
end

Prob{T<:AbstractFloat}(x::T) = Prob{T}(x)

import Base.*
*{T<:AbstractFloat}(x::Prob{T}, y::Prob{T}) = Prob{T}(float(x)*float(y))

In [ ]:
import Base: promote_rule, convert

promote_rule{S<:Real, T<:Real}(::Type{Prob{T}}, ::Type{S}) = Prob{promote_type(S, T)}

convert{S<:Real, T<:Real}(::Type{Prob{T}}, x::S) = Prob(float(x))
convert{T<:Real}(::Type{AbstractFloat}, x::Prob{T}) = x._


In [ ]:
x = Prob(0.5)


In [ ]:
3x

In [ ]:
promote(3, x)

These rules are always tricky to get right!

## `Vector`s of objects of parametric type

Suppose we make a vector of rationals, which is a parametric type

In [ ]:
v = [3//4, 4//5]

In [ ]:
typeof(v)

A **typealias**, that is, an alternative (shorter and/or more intuitive) name for the type `Array{T,1}` is `Vector{T}`:

In [ ]:
Array{Rational{Int64},1} === Vector{Rational{Int}}  

Here, the `===` operator checks for identity (rather than just equality) of objects:

In [ ]:
1 == 1.0

In [ ]:
1 === 1.0

We also have a subtype relationship:

In [ ]:
Rational{Int} <: Rational

where `Rational` is an abstract type, meaning that we cannot create objects of type `Rational`; they must always be parametrised.

**However**, we have

In [ ]:
Vector{Rational{Int}} <: Vector{Rational}

Thus if we define a method

In [ ]:
h(x::Vector{Rational}) = 2x

that accepts arguments that are `Vector`s of `Rational`s, we have

In [ ]:
v = [3//4, 4//5]
h(v)

i.e. `h` **does not** accept arguments of type `Vector{Rational{Int}}`. Instead, we must parametrize explicitly:

In [ ]:
h{T}(x::Vector{Rational{T}}) = 2x

In [ ]:
h(v)

Here, the syntax `{T}` means "define a function `h` with this template, for *any* type `T`.